In [55]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n):
    A = np.random.rand(n,n)
    return (A+A)
    
# Definitions
#A,b = np.array([[1.01,1.03],[0.97,1]]), np.array([2.01,1.99])
#A,b = np.array([[10,7,8,7],[7,6,7,5],[8,6,10,9],[7,5,9,10]]), np.array([32,23,33,31])


A = pos_def(5)
b = np.random.rand(5)
alpha = 0.01
M = A + alpha*np.identity(b.shape[0])
it = 100
error = 1e-6
print np.linalg.eig(A)

(array([ 3.88821176+0.j        ,  0.58746888+0.82125496j,
        0.58746888-0.82125496j,  0.53972977+0.j        , -0.14392942+0.j        ]), array([[ 0.50376669+0.j        ,  0.64984726+0.j        ,
         0.64984726-0.j        , -0.66310084+0.j        ,  0.38865215+0.j        ],
       [ 0.21106979+0.j        , -0.10003098+0.17141723j,
        -0.10003098-0.17141723j,  0.15270193+0.j        , -0.65593727+0.j        ],
       [ 0.23138203+0.j        , -0.01639747-0.44206072j,
        -0.01639747+0.44206072j, -0.00097798+0.j        ,  0.12886381+0.j        ],
       [ 0.55435775+0.j        ,  0.27394093-0.12590653j,
         0.27394093+0.12590653j,  0.72529061+0.j        ,  0.63121868+0.j        ],
       [ 0.58379664+0.j        , -0.39092238+0.31449248j,
        -0.39092238-0.31449248j, -0.10455608+0.j        , -0.06043921+0.j        ]]))


In [56]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
                
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [57]:
print "Normal GMRes:\n", spy.gmres(A,b, tol=1e-18), A.shape
print "Normal GMRes:\n", gmres(A,b, tol=1e-7),A.shape

Normal GMRes:
(array([ 2.42084779, -2.31368376,  0.73308821,  1.95414776, -0.30559634]), 50) (5, 5)
Normal GMRes:
(array([ 2.42085489, -2.31318715,  0.7329505 ,  1.95341046, -0.30555729]), 53) (5, 5)


In [58]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b,it=it,tol=tol)[0]
    #b = spy.gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        #w = gmres(M, np.dot(A,Q[:,i]), it=it, tol=tol)[0]
        w = spy.gmres(M, np.dot(A,Q[:,i]), tol=tol)[0]
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [59]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        #x = gmres(M, Q[:,i],it=it,tol=tol)[0]
        x = spy.gmres(M, Q[:,i], tol=tol)[0]
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    #return gmres(M, x_tild,it=it,tol=tol)[0], i
    return spy.gmres(M, x_tild, tol=tol)[0],i

In [60]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(a, b, N+1) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += myfun(i)
    int_val = myfun(0) + 2*int_val + myfun(x[-1])
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v,f, tol=1e-6):
    fz = f(z(t, (l+L)/2., (L-l)/2.))
    dzz = dz(t, (L-l)/2.)
    p = 1./(fz*dzz)
    #gmr,i = gmres(1./dzz *np.identity(v.shape[0]) - p*M, v, tol=tol)
    #gmr = np.linalg.solve(1./dzz *np.identity(v.shape[0]) - p*M, v)
    gmr = spy.gmres(1./dzz *np.identity(v.shape[0]) - p*M, v,tol=tol)[0]
    return gmr


def cauchy_integral(l, L, M, v, Nf = 50,N=1e2):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,M,v,f)
    val = trapezoid2(g1, N, 0, 2.*np.pi) / (2.*np.pi)
    return val

In [61]:
# GMRes using contour integral to compute the preconditioner
def cauchy_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = cauchy_integral(l, L, M, Q[:,i])
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    x = cauchy_integral(l, L, M, x_tild)
    return x,i

In [62]:
# GMRes using contour integral to compute the preconditioner
def cauchy_left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    b = cauchy_integral(l, L, M, b)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = cauchy_integral(l,L,M,np.dot(A,Q[:,i]))
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    return np.dot(Q[:,:i+1], y), i

In [63]:
print "Normal GMRes:\n", gmres(A,b,tol=error, it=it)
print "\nLeft Prec NC:\n", left_prec_gmres(A,b,M,tol=error, it=it)
print "\nRight Prec NC:\n", right_prec_gmres(A,b,M,tol=error, it=it)
print "\nNp Solve:\n", np.linalg.solve(A,b)
print "\nCauchy Right Prec:\n", cauchy_prec_gmres(A,b,M,it=it,tol=error)
print "\nCauchy Left Prec:\n", cauchy_left_prec_gmres(A, b, M, it=it, tol=error)

Normal GMRes:
(array([ 2.42376743, -2.31520046,  0.73344001,  1.95438821, -0.30585269]), 47)

Left Prec NC:
(array([ 2.42488588, -2.31838933,  0.73391231,  1.95786958, -0.30585836]), 6)

Right Prec NC:
(array([ 2.42409894, -2.317048  ,  0.73393742,  1.95725818, -0.3059326 ]), 9)

Np Solve:
[ 2.42084779 -2.31368376  0.73308821  1.95414776 -0.30559634]

Cauchy Right Prec:
(array([ 1.03090673 +1.28139867e-11j, -0.19914419 +5.17180042e-12j,
        0.34330854 +5.30904670e-13j, -0.30117028 +9.93032940e-12j,
        0.01463207 -1.21513500e-12j]), 99)

Cauchy Left Prec:
(array([ 1.16316345, -0.19085742,  0.31594574, -0.08863447, -0.11014909]), 3)
